In [1]:
from scipy import io
import pandas as pd
import numpy as np
from tqdm import trange

In [2]:
mat_file = io.loadmat('data/mpii_human_pose_v1_u12_1.mat', struct_as_record=False)

In [3]:
print(type(mat_file))
print(mat_file.keys())
print(mat_file)

<class 'dict'>
dict_keys(['__header__', '__version__', '__globals__', 'RELEASE'])
{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Tue Sep 23 22:09:02 2014', '__version__': '1.0', '__globals__': [], 'RELEASE': array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x7f09547e0490>]],
      dtype=object)}


In [4]:
release = mat_file['RELEASE']

In [5]:
release = release[0][0]

In [6]:
print(release._fieldnames)

['annolist', 'img_train', 'version', 'single_person', 'act', 'video_list']


In [7]:
annolist = release.__dict__['annolist']
img_tra = release.__dict__['img_train']
act = release.__dict__['act']
single_person = release.__dict__['single_person']
video_list = release.__dict__['video_list']
version = release.__dict__['version']
print(annolist.shape,img_tra.shape,act.shape, single_person.shape, video_list.shape, version.shape)

(1, 24987) (1, 24987) (24987, 1) (24987, 1) (1, 2821) (1,)


In [8]:
n = annolist.shape[1]

In [9]:
data_arr = ['NAME', 
            'r ankle_X',    'r ankle_Y',  '  r knee_X',     'r knee_Y', 
            'r hip_X',      'r hip_Y',      'l hip_X',      'l hip_Y',    
            'l knee_X',     'l knee_Y',     'l ankle_X',    'l ankle_Y',
            'pelvis_X',     'pelvis_Y',     'thorax_X',     'thorax_Y',
            'upper neck_X', 'upper neck_Y', 'head top_X',   'head top_Y', 
            'r wrist_X',    'r wrist_Y',    'r elbow_X',    'r elbow_Y',
            'r shoulder_X', 'r shoulder_Y', 'l shoulder_X', 'l shoulder_Y',
            'l elbow_X',    'l elbow_Y',    'l wrist_X',    'l wrist_Y',
            'Scale',        'Activity',     'Category']
print(data_arr)

['NAME', 'r ankle_X', 'r ankle_Y', '  r knee_X', 'r knee_Y', 'r hip_X', 'r hip_Y', 'l hip_X', 'l hip_Y', 'l knee_X', 'l knee_Y', 'l ankle_X', 'l ankle_Y', 'pelvis_X', 'pelvis_Y', 'thorax_X', 'thorax_Y', 'upper neck_X', 'upper neck_Y', 'head top_X', 'head top_Y', 'r wrist_X', 'r wrist_Y', 'r elbow_X', 'r elbow_Y', 'r shoulder_X', 'r shoulder_Y', 'l shoulder_X', 'l shoulder_Y', 'l elbow_X', 'l elbow_Y', 'l wrist_X', 'l wrist_Y', 'Scale', 'Activity', 'Category']


In [10]:
data = pd.DataFrame(columns=data_arr)

In [15]:
err = []

In [16]:
for ix in trange(0, annolist.shape[1]):
    if img_tra[0,ix] != 0:
        err.append('if img_tra[0,ix] != 0:')
        continue
    temp_arr = []
    obj_list = annolist[0,ix]
    obj_act = act[ix,0]
    
    rect = obj_list.__dict__['annorect']
    img_d = obj_list.__dict__['image']
    if rect.shape[0] == 0:
        err.append('if rect.shape[0] == 0:')
        continue
        
    obj_rect = rect[0,0]
    obj_img = img_d[0,0]
    
    
    if 'annopoints' not in obj_rect._fieldnames:
        err.append('if \'annopoints\' not in obj_rect._fieldnames:')
        continue
    
    name_d = obj_img.__dict__['name']
    name = name_d[0]
    temp_arr.append(name)
    annopoints = obj_rect.__dict__['annopoints']
    if annopoints.shape[0] == 0:
        err.append('if annopoints.shape[0] == 0:')
        continue
    obj_points = annopoints[0,0]
    points = obj_points.__dict__['point']
    cnt = 0
    px = 0
    
    for n in range(0,32):
        temp_arr.append(-1)
    
    
    for px in range(0,points.shape[1]):
        po = points[0,px]
        po_id = po.__dict__['id']
        
        po_x = po.__dict__['x']
        po_y = po.__dict__['y']
        ind = 2*po_id[0][0]+1
        temp_arr[ind] = po_x[0][0]
        temp_arr[ind+1] = po_y[0][0]
       
        
        
    scale = obj_rect.__dict__['scale']
    temp_arr.append(scale[0][0])
    
    activity = act[ix,0]
    
    a_n = activity.act_name
    c_n = activity.cat_name
    
    if a_n.shape[0]==0:
        temp_arr.append(a_n)
    else:
        temp_arr.append(activity.act_name[0])
    if c_n.shape[0]==0:
        temp_arr.append(c_n)
    else:
        temp_arr.append(activity.cat_name[0])
    
    temp_data_f = pd.DataFrame([temp_arr], columns=data_arr)
    
    data = pd.concat([data, temp_data_f]) 

100%|██████████| 24987/24987 [00:00<00:00, 392601.75it/s]


In [18]:
print(len(err))

24987


In [19]:
print(err[:10])

["if 'annopoints' not in obj_rect._fieldnames:", "if 'annopoints' not in obj_rect._fieldnames:", "if 'annopoints' not in obj_rect._fieldnames:", "if 'annopoints' not in obj_rect._fieldnames:", 'if img_tra[0,ix] != 0:', 'if img_tra[0,ix] != 0:', 'if img_tra[0,ix] != 0:', 'if img_tra[0,ix] != 0:', 'if img_tra[0,ix] != 0:', 'if img_tra[0,ix] != 0:']


In [12]:
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

In [13]:
print(data.shape)

(0, 36)


In [14]:
data.head(50)

,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,l knee_Y,l ankle_X,l ankle_Y,pelvis_X,pelvis_Y,thorax_X,thorax_Y,upper neck_X,upper neck_Y,head top_X,head top_Y,r wrist_X,r wrist_Y,r elbow_X,r elbow_Y,r shoulder_X,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category


In [17]:
data.to_csv('data/mpii_dataset_d.csv')